In [111]:
from pprint import pprint  # data pretty printer

from openfisca_core.taxbenefitsystems import TaxBenefitSystem

from openfisca_france.entities import entities

from openfisca_france import FranceTaxBenefitSystem

from openfisca_core.simulation_builder import SimulationBuilder


https://mybinder.org/repo/openfisca/tutorial
https://github.com/openfisca/tutorial/tree/master/notebooks


In [2]:
FranceTaxBenefitSystem()

https://github.com/openfisca/tutorial/blob/master/notebooks/how_to_handle_axes.ipynb

In [3]:
from openfisca_france import FranceTaxBenefitSystem
from openfisca_france.scenarios import init_single_entity

tax_benefit_system = FranceTaxBenefitSystem()  # Create an instance of French Tax and Benefit system

In [33]:
scenario = init_single_entity(
    tax_benefit_system.new_scenario(),
    
    # Axe declaration
    axes = [[
        dict(                       #  in a dictionary
            count = 100,            # 'count' : indicates the number of step
            min = 0,
            max = 100000,
            name = 'salaire_de_base', # the variable you want to make evolve
            ),
        ]],
    
    period = 2024,
    parent1 = dict(
        date_naissance = '1980-01-01',
    )
)

simulation = scenario.new_simulation()

In [34]:
simulation.calculate_add('salaire_de_base', 2024)


array([     0.     ,   1010.10114,   2020.2023 ,   3030.3027 ,
         4040.4045 ,   5050.5054 ,   6060.6055 ,   7070.707  ,
         8080.809  ,   9090.909  ,  10101.011  ,  11111.11   ,
        12121.211  ,  13131.315  ,  14141.414  ,  15151.515  ,
        16161.618  ,  17171.717  ,  18181.818  ,  19191.918  ,
        20202.021  ,  21212.121  ,  22222.22   ,  23232.326  ,
        24242.422  ,  25252.525  ,  26262.63   ,  27272.725  ,
        28282.828  ,  29292.932  ,  30303.03   ,  31313.13   ,
        32323.236  ,  33333.332  ,  34343.434  ,  35353.54   ,
        36363.637  ,  37373.74   ,  38383.836  ,  39393.938  ,
        40404.043  ,  41414.14   ,  42424.242  ,  43434.344  ,
        44444.44   ,  45454.547  ,  46464.652  ,  47474.75   ,
        48484.844  ,  49494.94   ,  50505.05   ,  51515.152  ,
        52525.26   ,  53535.363  ,  54545.45   ,  55555.55   ,
        56565.656  ,  57575.76   ,  58585.863  ,  59595.953  ,
        60606.06   ,  61616.16   ,  62626.26   ,  63636

In [37]:
income_tax = - simulation.calculate('ir_brut', 2024)
TMI = simulation.calculate('ir_tranche', 2024)
gross_wage = simulation.calculate_add('salaire_de_base', 2014)
taxable_income = simulation.calculate_add('salaire_imposable', 2014)
income_tax
#TMI

array([    -0.     ,     -0.     ,     -0.     ,     -0.     ,
           -0.     ,     -0.     ,     -0.     ,     -0.     ,
           -0.     ,     -0.     ,     -0.     ,     -0.     ,
           -0.     ,     -0.     ,     -0.     ,     -0.     ,
          -69.85247,   -151.82997,   -233.82208,   -315.82645,
         -397.81735,   -479.80862,   -561.8974 ,   -643.90155,
         -725.8928 ,   -807.87115,   -889.8759 ,   -971.85364,
        -1053.9556 ,  -1135.9725 ,  -1217.9379 ,  -1299.9293 ,
        -1381.9349 ,  -1463.9116 ,  -1546.027  ,  -1628.0187 ,
        -1710.0099 ,  -1791.9879 ,  -1873.9797 ,  -2008.8959 ,
        -2232.8076 ,  -2456.4578 ,  -2680.0703 ,  -2903.61   ,
        -3127.2588 ,  -3349.7722 ,  -3557.827  ,  -3784.3386 ,
        -4010.8232 ,  -4237.369  ,  -4463.8223 ,  -4690.3374 ,
        -4916.8164 ,  -5143.2964 ,  -5370.113  ,  -5596.592  ,
        -5823.0776 ,  -6049.5576 ,  -6276.0356 ,  -6502.552  ,
        -6729.107  ,  -6955.5493 ,  -7182.0317 ,  -7408

In [48]:
tax_benefit_system.parameters(2022).impot_revenu.bareme_ir_depuis_1945.bareme

  - threshold: 0
    rate: 0
  - threshold: 10777
    rate: 0.11
  - threshold: 27478
    rate: 0.3
  - threshold: 78570
    rate: 0.41
  - threshold: 168994
    rate: 0.45

In [62]:
tax_benefit_system.parameters(2024).impot_revenu.bareme_ir_depuis_1945.bareme

  - threshold: 0
    rate: 0
  - threshold: 11294
    rate: 0.11
  - threshold: 28797
    rate: 0.3
  - threshold: 82341
    rate: 0.41
  - threshold: 177106
    rate: 0.45

In [78]:
bareme2024 = tax_benefit_system.parameters.impot_revenu.bareme_ir_depuis_1945.bareme(2024)
type(bareme2024)
bareme2024.rates
bareme2024.thresholds = [0, 11294, 28797, 82341, 180000]
tax_benefit_system.parameters.impot_revenu.bareme_ir_depuis_1945.bareme(2024) = bareme2024.thresholds


SyntaxError: cannot assign to function call (1636975737.py, line 5)

https://github.com/openfisca/tutorial/blob/master/python/reforme_parametrique.py

In [105]:
from openfisca_core import reforms, periods
from openfisca_france.model.base import *

# Décrivez votre réforme

# Cette partie décrit les changements
def modifier_un_parametre(parameters):
    # Ceci décrit la periode sur laquelle va s'appliquer ce changement
    reform_year = 2024
    reform_period = periods.period(reform_year)
    # Cette partie propose un changement de taux pour le barème 1 (le second) de l'impôt sur le revenu à partir du début 2017
    #parameters.impot_revenu.bareme_ir_depuis_1945.bareme.thresholds[1].update(start=reform_period.start, value=11520)
    #parameters.impot_revenu.bareme_ir_depuis_1945.bareme.update(start=reform_period.start, value=11520)
    #parameters.impot_revenu.bareme_ir_depuis_1945.bareme[1].rate.update(start=reform_period.start, value=0.145)
    parameters.impot_revenu.bareme_ir_depuis_1945.bareme[1].threshold.update(start=reform_period.start, value=11520)
    parameters.impot_revenu.bareme_ir_depuis_1945.bareme[2].threshold.update(start=reform_period.start, value=29373)
    parameters.impot_revenu.bareme_ir_depuis_1945.bareme[3].threshold.update(start=reform_period.start, value=83988)
    parameters.impot_revenu.bareme_ir_depuis_1945.bareme[4].threshold.update(start=reform_period.start, value=180648)

    #parameters.impot_revenu.bareme_ir_depuis_1945.bareme[2].thresholds.update(start=reform_period.start, value=29373)
    #parameters.impot_revenu.bareme_ir_depuis_1945.bareme[3].thresholds.update(start=reform_period.start, value=83988)
    #parameters.impot_revenu.bareme_ir_depuis_1945.bareme[4].thresholds.update(start=reform_period.start, value=180648)
    return parameters

# Cette partie rassemble les changements de paramètre dans une seule réforme appelée ici MaReforme
class MaReforme(reforms.Reform):
    def apply(self):
        self.modify_parameters(modifier_function = modifier_un_parametre)

legislation_france = FranceTaxBenefitSystem()

resultat_actuel = legislation_france.parameters.impot_revenu.bareme_ir_depuis_1945.bareme(2024)

print("Résultat actuel")
print(resultat_actuel)

# Consultez la situation avec la reforme
legislation_reforme = MaReforme(legislation_france)

resultat_apres_reforme = legislation_reforme.parameters.impot_revenu.bareme_ir_depuis_1945.bareme(2024)

print("Resultat après reforme")
print(resultat_apres_reforme)

Résultat actuel
  - threshold: 0
    rate: 0
  - threshold: 11294
    rate: 0.11
  - threshold: 28797
    rate: 0.3
  - threshold: 82341
    rate: 0.41
  - threshold: 177106
    rate: 0.45
Resultat après reforme
  - threshold: 0
    rate: 0
  - threshold: 11520
    rate: 0.11
  - threshold: 29373
    rate: 0.3
  - threshold: 83988
    rate: 0.41
  - threshold: 180648
    rate: 0.45


In [116]:
TEST_CASE = {
    'individus': {
        'parent1': {
            'age': {'2015-01': 30},
            'salaire_de_base': {'2023': 200000,'2024': 200000}
        },
        'enfant1': {
            'age': {'2015-01': 12}
        },
        'enfant2': {
            'age': {'2015-01': 18}
        }
    }
}
TEST_CASE['foyers_fiscaux'] = {
    'foyer_fiscal1': {
        'declarants': ['parent1'],
        'personnes_a_charge': ['enfant1', 'enfant2'],
        'celibataire_ou_divorce': {'2023': True, '2024': True},
        'caseT': {'2023': True, '2024': True}
    }
}

TEST_CASE['menages'] = {
    'menage1': {
        'personne_de_reference': ['parent1'],
        'enfants': ['enfant1', 'enfant2']
    }
}

TEST_CASE['familles'] = {
    'famille1': {
        'parents': ['parent1'],
        'enfants': ['enfant1', 'enfant2']
    }
}

# display full test case
pprint(TEST_CASE)

{'familles': {'famille1': {'enfants': ['enfant1', 'enfant2'],
                           'parents': ['parent1']}},
 'foyers_fiscaux': {'foyer_fiscal1': {'caseT': {'2023': True, '2024': True},
                                      'celibataire_ou_divorce': {'2023': True,
                                                                 '2024': True},
                                      'declarants': ['parent1'],
                                      'personnes_a_charge': ['enfant1',
                                                             'enfant2']}},
 'individus': {'enfant1': {'age': {'2015-01': 12}},
               'enfant2': {'age': {'2015-01': 18}},
               'parent1': {'age': {'2015-01': 30},
                           'salaire_de_base': {'2023': 200000,
                                               '2024': 200000}}},
 'menages': {'menage1': {'enfants': ['enfant1', 'enfant2'],
                         'personne_de_reference': ['parent1']}}}


In [118]:
simulation_builder = SimulationBuilder()
simulation_2023 = simulation_builder.build_from_entities(legislation_france, TEST_CASE)
simulation_2024 = simulation_builder.build_from_entities(legislation_reforme, TEST_CASE)

print(simulation_2023.calculate('ir_plaf_qf', '2024'))
print(simulation_2023.calculate('ir_plaf_qf', '2023'))
print(simulation_2024.calculate('ir_plaf_qf', '2024'))
print(simulation_2024.calculate('ir_plaf_qf', '2023'))



[40386.008]
[40393.33]
[40070.54]
[40393.33]


https://legislation.fr.openfisca.org/?q=impot#search-input

In [31]:
type(tax_benefit_system.variables['assurance_vie_pfu_ir'])

4432265712_624334783936080248_ir_prelevement_forfaitaire_unique.assurance_vie_pfu_ir

In [56]:
scenario_qf = init_single_entity(
    tax_benefit_system.new_scenario(),
    
    # Axe declaration
    axes = [[
        dict(                       #  in a dictionary
            count = 100,            # 'count' : indicates the number of step
            min = 0,
            max = 100000,
            name = 'salaire_de_base', # the variable you want to make evolve
            ),
        ]],
    
    period = 2024,
    parent1 = dict(
        date_naissance = '1980-01-01',
    ),
    parent2 = dict(
        date_naissance = '1984-01-01',
    )
)
simulation_qf = scenario.new_simulation()
simulation_qf.calculate_add('salaire_de_base', 2024)
simulation_qf.calculate('ir_plaf_qf', 2024)
simulation_qf.calculate('nb_adult', 2024)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
      dtype=float32)